In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import csv
from scipy.stats import norm
import scipy.stats as stats
import statsmodels.api as sm
from patsy import dmatrices #a Python library for describing statistical models and building Design Matrices using R-like formulas.
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px 
from statsmodels.tsa.api import SimpleExpSmoothing
from statistics import stdev
from statistics import mean

In [2]:
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 21.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]

In [4]:
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-01-01','2022-04-01')['Adj Close'].index)

[*********************100%***********************]  1 of 1 completed


In [5]:
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,96.813492,76.819260,128.241989,111.874290,85.672462,182.556030,159.331497,147.581161,148.152328,115.185806,...,70.458305,231.919998,218.876419,37.469650,114.689148,341.345001,229.649994,215.285767,200.593628,37.906910
2021-01-05,97.814583,76.715622,128.280823,112.089272,88.364830,181.488495,160.925507,147.154129,148.474213,115.800583,...,71.140762,234.910004,217.072510,38.086960,114.787102,336.754608,235.570007,212.072693,199.693878,39.733753
2021-01-06,96.969612,77.327995,133.911011,114.639877,89.577888,191.901703,156.919006,151.101746,154.150620,120.084435,...,75.355347,226.830002,229.939713,38.714066,119.802994,350.887024,256.549988,210.204163,197.805405,40.747643
2021-01-07,98.007462,75.264778,135.260315,117.659554,93.256454,194.221130,158.108002,149.042526,158.958969,119.134567,...,75.023354,235.039993,231.590073,39.360771,119.538467,356.512573,264.679993,211.380630,198.705139,41.067345
2021-01-08,98.521782,74.595879,134.551682,117.249107,94.242668,195.599182,159.134995,150.352081,152.882690,119.252083,...,74.423889,242.460007,229.296844,39.978077,119.920547,354.910889,260.149994,213.001999,198.635925,41.524048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,156.891495,95.333153,183.029373,111.363403,136.376923,311.729828,164.773499,241.656281,171.694946,188.315094,...,118.004997,113.760002,295.426666,33.621418,89.567093,509.607941,363.670013,217.628143,207.255630,83.524574
2022-03-28,157.513885,96.091171,182.225754,112.533562,136.973938,308.232391,168.990494,244.344177,172.051941,187.741089,...,117.223892,116.169998,294.201508,34.228012,90.897392,509.776794,363.399994,219.959564,210.842758,81.181572
2022-03-29,157.718109,97.154373,179.884308,114.090469,140.735092,309.342041,169.315002,252.427429,177.406876,188.968292,...,118.004997,121.180000,298.953888,34.953945,94.858521,507.323273,365.200012,227.282562,214.031296,80.750229


In [7]:
rd1= rd.pct_change().shift(-1)
rd1=rd1.dropna()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,0.010340,-0.001349,0.000303,0.001922,0.031426,-0.005848,0.010004,-0.002894,0.002173,0.005337,...,0.009686,0.012892,-0.008242,0.016475,0.000854,-0.013448,0.025778,-0.014925,-0.004485,0.048193
2021-01-05,-0.008638,0.007982,0.043890,0.022755,0.013728,0.057377,-0.024897,0.026826,0.038232,0.036993,...,0.059243,-0.034396,0.059276,0.016465,0.043697,0.041967,0.089060,-0.008811,-0.009457,0.025517
2021-01-06,0.010703,-0.026681,0.010076,0.026341,0.041066,0.012087,0.007577,-0.013628,0.031193,-0.007910,...,-0.004406,0.036194,0.007177,0.016705,-0.002208,0.016032,0.031690,0.005597,0.004549,0.007846
2021-01-07,0.005248,-0.008887,-0.005239,-0.003488,0.010575,0.007095,0.006496,0.008786,-0.038225,0.000986,...,-0.007990,0.031569,-0.009902,0.015683,0.003196,-0.004493,-0.017115,0.007670,-0.000348,0.011121
2021-01-08,0.016314,-0.005052,-0.005772,0.002917,0.025115,0.000149,-0.021519,-0.006690,0.029282,-0.005912,...,0.000992,-0.020457,-0.007198,0.002696,-0.012581,-0.007265,-0.010379,-0.011882,-0.015082,0.030356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,0.006551,0.004148,0.008804,-0.002576,-0.011040,0.009335,0.006868,0.016612,0.037885,0.007359,...,0.014266,-0.019733,0.013593,0.008050,-0.002212,-0.000837,0.013065,0.005154,-0.001153,0.021828
2022-03-25,0.003967,0.007951,-0.004391,0.010508,0.004378,-0.011219,0.025593,0.011123,0.002079,-0.003048,...,-0.006619,0.021185,-0.004147,0.018042,0.014853,0.000331,-0.000742,0.010713,0.017308,-0.028052
2022-03-28,0.001297,0.011065,-0.012849,0.013835,0.027459,0.003600,0.001920,0.033081,0.031124,0.006537,...,0.006663,0.043126,0.016153,0.021209,0.043578,-0.004813,0.004953,0.033292,0.015123,-0.005313


In [12]:
def pct_for2(st,i,title):
  print(i)
  insample=st.iloc[:-i]
  
  outsample=st.iloc[-i:]
  
  insample.index=pd.to_datetime(insample.index)
  outsample.index=pd.to_datetime(outsample.index)
  print('auto einai to insample',insample)
  print('auto einai to outsample',outsample)
  pred = outsample.copy()
  # print(insample.values)
  model = SimpleExpSmoothing(np.asarray(insample.values))
  model.index = pd.to_datetime(insample.index)
  # print(model)
  fit1 = model.fit()
  # print(fit1.fittedvalues)
  pred1=fit1.forecast(1)
  # print(pred1)
  # print("alpha= ",fit1.params['smoothing_level'])
  # print(insample.values)
  # print(st.index[-i])
  dif=np.subtract(insample.values,fit1.fittedvalues)
  # print(dif)
  me=dif.mean()
  sd=dif.std()
  print("prediction->",pred1[0])
  print("mean->",me)
  print("sd->",sd)
  x= np.arange((pred1[0]+me)-3*sd,(pred1[0]+me)+ 3*sd,0.001)
  #print(x)
  pdf= stats.norm.pdf(x, loc=me+pred1, scale=sd)
  plt.plot(x,pdf,label=title)
  return pred1+me,sd

In [13]:
def cdf(r1,r2,r3,r4,mean,sd):
  temp=[]
  #print(mean,sd)
  p1=norm(loc = mean , scale =sd ).cdf(r1)
  p2=norm(loc = mean , scale =sd ).cdf(r2)-p1
  p3=norm(loc = mean , scale =sd ).cdf(r3)-p1-p2
  p4=norm(loc = mean , scale =sd ).cdf(r4)-p1-p2-p3
  p5=1-p1-p2-p3-p4
  temp.append(p1)
  temp.append(p2)
  temp.append(p3)
  temp.append(p4)
  temp.append(p5)
  return temp

In [14]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    # min_date = min(hist_data.date)
    # max_date = max(hist_data.date)

    #alternative for point forecast
    min_date = min(hist_data.date)
    max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [18]:
from collections import OrderedDict
def pct_vis2(data):
  final=dict()
  for i in range(253,len(data.index)-1,1):
    print('i=========',data.index[i])
    day=[]
    d=data.loc[data.index[i]] #για κάθε μέρα μετά τις πρώτες 253 μέρες (χρειάζονται για το insample)
    plt.figure(figsize=(18,10))
    plt.title(data.index[i])
    temp=dict()
    for j in range(len(d.index)):#για κάθε μετοχή 
      # print(d.index[j],'kai',i,'kai',data[d.index[j]])
      x,y=pct_for2(data[d.index[j]],i,d.index[j])
      print(x,y)
      temp[x[0]]=y #θα κρατάω σε ένα dict ta means και τα sd των κατανομών 
    res = list(temp.keys())#εδώ θα έχω όλα τα keys
    # temp2=dict(sorted(temp.items()))#εδώ θα έχω τα means sorted των κατανομών σε αύξουσα σειρά (λίστα)
    temp2=sorted(temp.keys())
    print('temp2',temp2)
    R1=temp2[8]#To δέκατο στοιχείο
    R2=temp2[18]#Το 20ο κ.λ.π.
    R3=temp2[27]
    R4=temp2[36]
    print(R1,R2,R3,R4)
  
    #φτιάχνω μια συνάρτηση τώρα που δίνω τις περιοχές και τις τιμές κάθε μετοχής  και μου επιστρέφει έναν πίνακα με τις πιθανότητες
    for k in range(len(d.index)):
      #print(k)
      js=cdf(R1,R2,R3,R4,res[k],temp[res[k]]) #επιστρέφει έναν πίνακα με τα ranks κάθε μετοχής για εκείνη τη μέρα
      #print(js,d.index[k])
      day.append(js)
    plt.legend()
    plt.ylim(0)
    final[data.index[i]]=day
    
  #print(len(final))
  return final
final=pct_vis2(rd1)

Output hidden; open in https://colab.research.google.com to view.

In [20]:
t2=pd.DataFrame.from_dict(final).T
t2.columns=assets
t2=t2.rename_axis('date')
t2=t2.reset_index()
t2

,date,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
0,2022-01-04,"[0.4718948929454726, 0.03080035344925769, 0.02...","[0.4609048972524892, 0.039095102747510824, 0.0...","[0.45946088749879443, 0.027809976511786072, 0....","[0.4518549806001098, 0.027758736103271175, 0.0...","[0.4141324979570515, 0.012389645110628345, 0.0...","[0.41658276258184285, 0.02413637707757954, 0.0...","[0.5001090218994125, 0.029373203602808373, 0.0...","[0.40246526530903576, 0.034033297373133486, 0....","[0.42611572786987945, 0.023570409002605708, 0....",...,"[0.4212273751520826, 0.025683609638454286, 0.0...","[0.4794325140174263, 0.015646415002882996, 0.0...","[0.4547290534548981, 0.02767279466617728, 0.02...","[0.5241195959107069, 0.024847102206104643, 0.0...","[0.4815652242172247, 0.021488986936181176, 0.0...","[0.453714830110168, 0.03646181125540554, 0.026...","[0.40651907849404767, 0.016506603387905272, 0....","[0.4985328388871698, 0.027543505999270468, 0.0...","[0.5385985234449118, 0.026548419091727893, 0.0...","[0.39937125663461703, 0.02056986248571291, 0.0..."
1,2022-01-05,"[0.4800434933847417, 0.028583163158277358, 0.0...","[0.4639226226079311, 0.0360773773920689, 0.038...","[0.4532259739098872, 0.025967124106479278, 0.0...","[0.45497190882822464, 0.025620199872696092, 0....","[0.4228498799936175, 0.011618752269818011, 0.0...","[0.41751976593479195, 0.02228047352382473, 0.0...","[0.5079867604705338, 0.02722198135956866, 0.02...","[0.39610580070225565, 0.03173867069525127, 0.0...","[0.42473947824337677, 0.021785913159386272, 0....",...,"[0.4187466102993641, 0.023773022796697352, 0.0...","[0.4863655505607232, 0.014522733213874373, 0.0...","[0.44602772821068565, 0.02601110466848433, 0.0...","[0.5318507043600504, 0.023020160499914954, 0.0...","[0.4821938380536831, 0.0198310513429939, 0.021...","[0.4545617711450765, 0.033678633479437226, 0.0...","[0.4042588835551799, 0.015254842324394724, 0.0...","[0.500717855853246, 0.025407931132986006, 0.02...","[0.5429369880461818, 0.024489666507586993, 0.0...","[0.39540489365873266, 0.019053669209235125, 0...."
2,2022-01-06,"[0.4842879275426568, 0.03141706805030453, 0.03...","[0.4614241695281601, 0.040072230728823044, 0.0...","[0.4591468197081906, 0.028596943937350294, 0.0...","[0.45517449468216564, 0.028238859226565605, 0....","[0.4253536386734974, 0.012787654150252736, 0.0...","[0.425009540871512, 0.02464915990672062, 0.027...","[0.5093081230262674, 0.029941208669902797, 0.0...","[0.40385086496350747, 0.0351029202672955, 0.03...","[0.4319441370218029, 0.02408237450334194, 0.02...",...,"[0.42694103575115544, 0.026328914020323002, 0....","[0.4891794880678736, 0.01596433467571101, 0.01...","[0.44966131573737605, 0.02862313782125453, 0.0...","[0.5328646889561488, 0.025304371208989207, 0.0...","[0.48580985732317694, 0.021800724317097342, 0....","[0.3307666269119591, 0.03485614329786435, 0.04...","[0.4115080738170342, 0.0169117160386692, 0.018...","[0.4997577120353489, 0.028032452345326142, 0.0...","[0.5383633024267704, 0.02728982124380952, 0.03...","[0.39318568916702795, 0.021039298971822273, 0...."
3,2022-01-07,"[0.48548724941317806, 0.031667378145671055, 0....","[0.4642793191568527, 0.04052609926670714, 0.05...","[0.4547375955372302, 0.028810309287870595, 0.0...","[0.4517461059234477, 0.028427720858077432, 0.0...","[0.4195610493984151, 0.012905521605841463, 0.0...","[0.4176242586452267, 0.024907617136771587, 0.0...","[0.510996791348785, 0.030191370224928016, 0.04...","[0.39587573195734244, 0.03544037453686333, 0.0...","[0.4278166406670453, 0.02424966601530487, 0.03...",...,"[0.41894247940090223, 0.02662910161629284, 0.0...","[0.48320129146878216, 0.016141023236217134, 0....","[0.4536444569797281, 0.028991170166838998, 0.0...","[0.5316528097672444, 0.025477695349517493, 0.0...","[0.47858907852030974, 0.02207995710502908, 0.0...","[0.33937597003654846, 0.035407538618412226, 0....","[0.4055088043710723, 0.017059744832169543, 0.0...","[0.5, 0.028230238278329822, 0.039959433410316

In [21]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [22]:
RPS=pd.DataFrame()

# RPS=RPS.drop('date',axis=1)
# print(RPS.index[2])
for k in t2.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(t2['date'][k]!=t2['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==t2['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)              
print(RPS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == dat

          0         1         2         3         4         5         6   \
0   0.214670  0.216166  0.182459  0.187599  0.257980  0.167144  0.179912   
1   0.172538  0.169727  0.185800  0.203540  0.156326  0.167669  0.164743   
2   0.176507  0.219374  0.202671  0.184587  0.246837  0.172627  0.179254   
3   0.233104  0.164969  0.198611  0.182163  0.217770  0.178461  0.159574   
4   0.169774  0.188752  0.178878  0.180418  0.172284  0.166625  0.242500   
5   0.178952  0.179766  0.188042  0.185665  0.156599  0.169639  0.170006   
6   0.181711  0.208840  0.186515  0.182369  0.215522  0.169418  0.175821   
7   0.222469  0.184464  0.179226  0.181680  0.158828  0.164968  0.198820   
8   0.234074  0.182540  0.176599  0.183422  0.240016  0.241898  0.163424   
9   0.166702  0.185052  0.176463  0.176155  0.240253  0.241434  0.159979   
10  0.169288  0.187719  0.179820  0.200942  0.249496  0.179240  0.168887   
11  0.177206  0.224020  0.176096  0.204887  0.216745  0.232692  0.163358   
12  0.172400

In [23]:
d=t2['date']
name=t2.columns[:-1]
# print(np.split(name.values,len(name)))
# print(d.values)
RPS['date']=d.values
RPS=RPS.set_index('date')
RPS.columns=assets
RPS

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.214670,0.216166,0.182459,0.187599,0.257980,0.167144,0.179912,0.176529,0.170386,0.170014,...,0.203368,0.190190,0.179489,1.553500e-01,0.190253,0.180141,0.259505,0.180907,0.143796,0.148114
2022-01-05,0.172538,0.169727,0.185800,0.203540,0.156326,0.167669,0.164743,0.174515,0.182135,0.169362,...,0.167879,0.208314,0.191706,2.596584e-01,0.182043,0.194118,0.190471,0.169061,0.140302,0.146079
2022-01-06,0.176507,0.219374,0.202671,0.184587,0.246837,0.172627,0.179254,0.198443,0.181819,0.173687,...,0.175049,0.189785,0.195193,2.188246e-01,0.185807,0.236697,0.253721,0.168555,0.142365,0.162925
2022-01-07,0.233104,0.164969,0.198611,0.182163,0.217770,0.178461,0.159574,0.169356,0.226615,0.177844,...,0.176686,0.192221,0.183392,2.187170e-01,0.188364,0.129752,0.218606,0.165127,0.224278,0.184884
2022-01-10,0.169774,0.188752,0.178878,0.180418,0.172284,0.166625,0.242500,0.205332,0.170988,0.182729,...,0.181806,0.208235,0.178738,1.508529e-01,0.212643,0.172508,0.152981,0.202128,0.185600,0.143390
2022-01-11,0.178952,0.179766,0.188042,0.185665,0.156599,0.169639,0.170006,0.158983,0.172367,0.203116,...,0.182549,0.194463,0.203379,2.553075e-01,0.183787,0.202319,0.250992,0.201359,0.143871,0.227396
2022-01-12,0.181711,0.208840,0.186515,0.182369,0.215522,0.169418,0.175821,0.158116,0.184617,0.183679,...,0.169971,0.202083,0.199439,1.648609e-01,0.185136,0.179323,0.157599,0.182491,0.145114,0.215599
2022-01-13,0.222469,0.184464,0.179226,0.181680,0.158828,0.164968,0.198820,0.176513,0.203476,0.233882,...,0.164966,0.191122,0.200229,1.532508e-01,0.191782,0.181070,0.255359,0.181306,0.136713,0.142071
2022-01-14,0.234074,0.182540,0.176599,0.183422,0.240016,0.241898,0.163424,0.158286,0.168819,0.180124,...,0.179833,0.188014,0.198489,1.593805e-01,0.192709,0.170260,0.259176,0.243563,0.286376,0.139759


In [24]:
RPS.mean(axis=1).mean()

0.19278034684158302

In [ ]:
#save results
RPS.to_excel('Method_3-SES_in_perc_returns.xlsx')

#####Compare with point forecast 
***

In [ ]:
t2.set_index('date',inplace=True)

In [ ]:
flag=False
for i in t2: #for every stock
  print(i)
  for j in range(len(t2[i].values)): #each day of this stock
    temp = t2[i].values[j]
    max= np.max(temp) #find max possiblity
    #replace this max possibility with 1
    temp2=np.where(temp!=max,0,temp)
    temp2=np.where(temp2==max,1,temp2)
    print(temp,'kai',temp2)
    if((temp2==1).sum()>1):
      print('isotimia')
      flag=True
      break
    t2[i].values[j]=temp2

In [ ]:
t2=t2.reset_index()

In [ ]:
flag

False

In [ ]:
RPS=pd.DataFrame()

# RPS=RPS.drop('date',axis=1)
# print(RPS.index[2])
for k in t2.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(t2['date'][k]!=t2['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==t2['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    print(np.max(hist_data.date))
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    
    print(hist_data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)              
print(RPS)

[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
2022-02-04
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


   symbol        date       price
0    ABBV  2022-03-30  160.865295
1     AEP  2022-03-30   97.873009
2     AIZ  2022-03-30  180.737549
3    ALLE  2022-03-30  112.891090
4    AMAT  2022-03-30  135.123199
..    ...         ...         ...
40    UNH  2022-03-31  508.157806
41    URI  2022-03-31  355.209991
42      V  2022-03-31  220.955887
43   VRSK  2022-03-31  214.218460
44    XOM  2022-03-31   80.965904

[90 rows x 3 columns]
     0    1    2    3    4    5    6    7    8    9   ...   35   36   37   38  \
0   0.0  0.8  0.6  0.2  0.8  0.2  0.4  0.4  0.2  0.2  ...  0.6  0.2  0.4  0.0   
1   0.6  0.2  0.2  0.0  0.0  0.2  0.2  0.4  0.4  0.2  ...  0.2  0.8  0.8  0.8   
2   0.4  0.8  0.8  0.2  0.8  0.2  0.4  0.6  0.4  0.2  ...  0.0  0.0  0.8  0.6   
3   0.0  0.4  0.0  0.2  0.6  0.4  0.2  0.4  0.8  0.4  ...  0.4  0.0  0.6  0.6   
4   0.6  0.0  0.4  0.6  0.2  0.0  0.8  0.6  0.0  0.4  ...  0.4  0.8  0.6  0.0   
5   0.4  0.6  0.2  0.2  0.0  0.2  0.2  0.2  0.2  0.6  ...  0.4  0.0  0.0  0.8   
6 

In [ ]:
d=t2['date']
name=t2.columns[:-1]
# print(np.split(name.values,len(name)))
# print(d.values)
RPS['date']=d.values
RPS=RPS.set_index('date')
RPS.columns=assets
RPS

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.0,0.8,0.6,0.2,0.8,0.2,0.4,0.4,0.2,0.2,...,0.6,0.2,0.4,0.0,0.0,0.2,0.8,0.4,0.0,0.0
2022-01-05,0.6,0.2,0.2,0.0,0.0,0.2,0.2,0.4,0.4,0.2,...,0.2,0.8,0.8,0.8,0.2,0.8,0.4,0.2,0.0,0.0
2022-01-06,0.4,0.8,0.8,0.2,0.8,0.2,0.4,0.6,0.4,0.2,...,0.0,0.0,0.8,0.6,0.0,0.6,0.8,0.2,0.0,0.2
2022-01-07,0.0,0.4,0.0,0.2,0.6,0.4,0.2,0.4,0.8,0.4,...,0.4,0.0,0.6,0.6,0.0,0.0,0.6,0.0,0.6,0.4
2022-01-10,0.6,0.0,0.4,0.6,0.2,0.0,0.8,0.6,0.0,0.4,...,0.4,0.8,0.6,0.0,0.8,0.4,0.0,0.6,0.4,0.0
2022-01-11,0.4,0.6,0.2,0.2,0.0,0.2,0.2,0.2,0.2,0.6,...,0.4,0.0,0.0,0.8,0.0,0.0,0.8,0.6,0.0,0.6
2022-01-12,0.8,0.8,0.2,0.2,0.6,0.2,0.0,0.2,0.4,0.4,...,0.0,0.0,0.8,0.2,0.4,0.6,0.0,0.4,0.0,0.4
2022-01-13,0.0,0.6,0.4,0.2,0.0,0.2,0.6,0.4,0.6,0.8,...,0.0,0.2,0.8,0.0,0.0,0.2,0.8,0.4,0.0,0.0
2022-01-14,0.0,0.6,0.4,0.6,0.8,0.8,0.2,0.2,0.2,0.4,...,0.4,0.2,0.0,0.2,0.0,0.4,0.8,0.8,0.8,0.0


In [ ]:
RPS.mean(axis=1).mean()

0.39037037037037037

In [ ]:
#save results
RPS.to_excel('Method_3-SES_in_perc_returns-POINT_FORECAST.xlsx')